In [1]:
import pandas as pd
import json
import re
import numpy as np
import sys
from SARAH import *

In [2]:
SPfull={'Properties':{
         'OnlyLowEnergySPheno' : True,
         'AddTreeLevelUnitarityLimits':True,
        'MINPAR':[[1,'LambdaIN']],
        'ParametersToSolveTadpoles':['mu2'],
        'BoundaryLowScaleInput':[['\[Lambda]','LambdaIN']],
        'ListDecayParticles':['Fu','Fe','Fd','hh'],
        'ListDecayParticles3B': [['Fu','"Fu.f90"'],['Fe','"Fe.f90"'],['Fd','"Fd.f90"']],
        'RenConditionsDecays':[
               ['dCosTW', '1/2*Cos[ThetaW] * (PiVWp/(MVWp^2) - PiVZ/(mVZ^2))'],
               ['dSinTW', '-dCosTW/Tan[ThetaW]'],
               ['dg2', '1/2*g2*(derPiVPheavy0 + PiVPlightMZ/MVZ^2 - (-(PiVWp/MVWp^2) + PiVZ/MVZ^2)/Tan[ThetaW]^2 + (2*PiVZVP*Tan[ThetaW])/MVZ^2)'],
               ['dg1', 'dg2*Tan[ThetaW]+g2*dSinTW/Cos[ThetaW]- dCosTW*g2*Tan[ThetaW]/Cos[ThetaW]']
             ],
        'DEFINITION': {'MatchingConditions':
                 [['v' ,'vSM'], 
                  ['Ye', 'YeSM'],
                  ['Yd', 'YdSM'],
                  ['Yu', 'YuSM'],
                  ['g1', 'g1SM'],
                  ['g2', 'g2SM'],
                  ['g3', 'g3SM']]},
        'DefaultInputValues' :['LambdaIN' , 0.27]
           },
    'Index':{'OnlyLowEnergySPheno':0,'MINPAR':1, 'ParametersToSolveTadpoles':2, 
                'BoundaryLowScaleInput':3, 'DEFINITION':4, 'ListDecayParticles':5, 
                'ListDecayParticles3B':6, 'DefaultInputValues':7, 
                'AddTreeLevelUnitarityLimits':8, 'RenConditionsDecays':9}   
   }
SP={'Properties':{
         'OnlyLowEnergySPheno' : True,
         'AddTreeLevelUnitarityLimits':True,
        'MINPAR':[],
        'ParametersToSolveTadpoles':[],
        'BoundaryLowScaleInput':[],
        'ListDecayParticles':[],
        'ListDecayParticles3B': [],
        'RenConditionsDecays':[
               ['dCosTW', '1/2*Cos[ThetaW] * (PiVWp/(MVWp^2) - PiVZ/(mVZ^2))'],
               ['dSinTW', '-dCosTW/Tan[ThetaW]'],
               ['dg2', '1/2*g2*(derPiVPheavy0 + PiVPlightMZ/MVZ^2 - (-(PiVWp/MVWp^2) + PiVZ/MVZ^2)/Tan[ThetaW]^2 + (2*PiVZVP*Tan[ThetaW])/MVZ^2)'],
               ['dg1', 'dg2*Tan[ThetaW]+g2*dSinTW/Cos[ThetaW]- dCosTW*g2*Tan[ThetaW]/Cos[ThetaW]']
             ],
        'DEFINITION': {'MatchingConditions':
                 []},
        'DefaultInputValues' :{}
           },
    'Index':{'OnlyLowEnergySPheno':0,'MINPAR':1, 'ParametersToSolveTadpoles':2, 
                'BoundaryLowScaleInput':3, 'DEFINITION':4, 'ListDecayParticles':5, 
                'ListDecayParticles3B':6, 'DefaultInputValues':7, 
                'AddTreeLevelUnitarityLimits':8, 'RenConditionsDecays':9}   
   }

## Local definitions

## Example dictionary

In [3]:
SPdf=pd.DataFrame(SPfull)

In [4]:
SPdf

,Index,Properties
AddTreeLevelUnitarityLimits,8,True
BoundaryLowScaleInput,3,"[[\[Lambda], LambdaIN]]"
DEFINITION,4,"{'MatchingConditions': [['v', 'vSM'], ['Ye', '..."
DefaultInputValues,7,"[LambdaIN, 0.27]"
ListDecayParticles,5,"[Fu, Fe, Fd, hh]"
ListDecayParticles3B,6,"[[Fu, ""Fu.f90""], [Fe, ""Fe.f90""], [Fd, ""Fd.f90""]]"
MINPAR,1,"[[1, LambdaIN]]"
OnlyLowEnergySPheno,0,True
ParametersToSolveTadpoles,2,[mu2]
RenConditionsDecays,9,"[[dCosTW, 1/2*Cos[ThetaW] * (PiVWp/(MVWp^2) - ..."


## Filling real dictionary

save dictionary

In [5]:
f=open('sp.json','w')
json.dump(SP,f)
f.close()

load parameters

In [6]:
f=open('fullparamenter.json','r')
fp=json.load(f)
f.close()

In [7]:
smd=pd.DataFrame( fp['smdict'] )
smc=pd.DataFrame( fp['coupling'] )

In [8]:
smd.loc['Lorentz']=smd.loc['Lorentz'].apply(lambda l: [] if type(l)!=list else l)

In [9]:
dimL=smd.loc['Lorentz'].apply(lambda l: len(l) )

In [10]:
L=smd.loc['Lorentz'].apply(lambda l: len( list(np.unique(l))) )

In [11]:
sci=get_input_parameters(L,dimL,smd)

In [12]:
sciIN=get_input_parameters_IN(sci,suffix='IN')

In [13]:
DP,DP3B=get_decay_particles(DecayParticles   = ['Fu', 'Fe', 'Fd', 'hh'],
                    DecayParticles3B = ['Fu', 'Fe', 'Fd'])

In [14]:
get_other_yukawas(smd,L,dimL)

[]

In [15]:
SP['Properties']['ParametersToSolveTadpoles']=get_tadpoles_and_bilinears(L,dimL,smd)[0]
SP['Properties']['ParametersToSolveTadpoles']

['mu2']

In [16]:
SP['Properties']['BoundaryLowScaleInput']=get_BoundaryLowScaleInput(sci,sciIN)
SP['Properties']['BoundaryLowScaleInput']

[['\\[Lambda]', 'LambdaIN']]

In [17]:
SP['Properties']['MINPAR']=get_MINPAR(sciIN)
SP['Properties']['MINPAR']

[[1, 'LambdaIN']]

In [18]:
SP['Properties']['DEFINITION']['MatchingConditions']=get_SM_MatchingConditions(smd,smc)
SP['Properties']['DEFINITION']['MatchingConditions']

[['v', 'vSM'],
 ['Yd', 'YdSM'],
 ['Ye', 'YeSM'],
 ['Yu', 'YuSM'],
 ['g1', 'g1SM'],
 ['g2', 'g2SM'],
 ['g3', 'g3SM']]

In [19]:
SP['Properties']['ListDecayParticles']=DP
SP['Properties']['ListDecayParticles3B']=DP3B

In [20]:
SP['Properties']['DefaultInputValues'].update( 
                     get_sm_DefaultInputValues(smd,sci,sciIN) )
SP['Properties']['DefaultInputValues']

{'LambdaIN': 0.27}

###  Filled dictionary

In [21]:
SP

{'Index': {'AddTreeLevelUnitarityLimits': 8,
  'BoundaryLowScaleInput': 3,
  'DEFINITION': 4,
  'DefaultInputValues': 7,
  'ListDecayParticles': 5,
  'ListDecayParticles3B': 6,
  'MINPAR': 1,
  'OnlyLowEnergySPheno': 0,
  'ParametersToSolveTadpoles': 2,
  'RenConditionsDecays': 9},
 'Properties': {'AddTreeLevelUnitarityLimits': True,
  'BoundaryLowScaleInput': [['\\[Lambda]', 'LambdaIN']],
  'DEFINITION': {'MatchingConditions': [['v', 'vSM'],
    ['Yd', 'YdSM'],
    ['Ye', 'YeSM'],
    ['Yu', 'YuSM'],
    ['g1', 'g1SM'],
    ['g2', 'g2SM'],
    ['g3', 'g3SM']]},
  'DefaultInputValues': {'LambdaIN': 0.27},
  'ListDecayParticles': ['Fu', 'Fe', 'Fd', 'hh'],
  'ListDecayParticles3B': [['Fu', '"Fu.f90"'],
   ['Fe', '"Fe.f90"'],
   ['Fd', '"Fd.f90"']],
  'MINPAR': [[1, 'LambdaIN']],
  'OnlyLowEnergySPheno': True,
  'ParametersToSolveTadpoles': ['mu2'],
  'RenConditionsDecays': [['dCosTW',
    '1/2*Cos[ThetaW] * (PiVWp/(MVWp^2) - PiVZ/(mVZ^2))'],
   ['dSinTW', '-dCosTW/Tan[ThetaW]'],
   ['dg2

## Output file

In [22]:
f=open('SPheno.json','w')
json.dump(SP,f)
f.close()

In [23]:
SP=pd.read_json('SPheno.json')
SP=SP.sort_values('Index')

In [24]:
to_SPheno(SP,'sp.m',dictentries=['DefaultInputValues'])

In [25]:
cat sp.m

OnlyLowEnergySPheno = True;

MINPAR = {{1, LambdaIN}};

ParametersToSolveTadpoles = {mu2};

BoundaryLowScaleInput = {{\[Lambda], LambdaIN}};

DEFINITION[MatchingConditions]={{v, vSM}, {Yd, YdSM}, {Ye, YeSM}, {Yu, YuSM}, {g1, g1SM}, {g2, g2SM}, {g3, g3SM}};

ListDecayParticles = {Fu, Fe, Fd, hh};

ListDecayParticles3B = {{Fu, "Fu.f90"}, {Fe, "Fe.f90"}, {Fd, "Fd.f90"}};

DefaultInputValues = {LambdaIN -> 0.27};

AddTreeLevelUnitarityLimits = True;

RenConditionsDecays = {{dCosTW, 1/2*Cos{ThetaW} * (PiVWp/(MVWp^2) - PiVZ/(mVZ^2))}, {dSinTW, -dCosTW/Tan{ThetaW}}, {dg2, 1/2*g2*(derPiVPheavy0 + PiVPlightMZ/MVZ^2 - (-(PiVWp/MVWp^2) + PiVZ/MVZ^2)/Tan{ThetaW}^2 + (2*PiVZVP*Tan{ThetaW})/MVZ^2)}, {dg1, dg2*Tan{ThetaW}+g2*dSinTW/Cos{ThetaW}- dCosTW*g2*Tan{ThetaW}/Cos{ThetaW}}};

